In [1]:
! pip install -r requirements.txt

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


In [2]:
from paddlets.datasets.repository import get_dataset
from paddlets.models.forecasting.ml.ml_model_wrapper import make_ml_model

from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor


In [3]:
import pandas as pd
sunshine = pd.read_csv("../data/sunshine.csv")
sunshine.describe()

,Day,Hour,Radiation
count,4500.000000,4500.000000,4500.000000
mean,149.500000,13.000000,0.253720
std,86.611683,4.320974,0.264251
min,0.000000,6.000000,0.000000
25%,74.750000,9.000000,0.014409
50%,149.500000,13.000000,0.161383
75%,224.250000,17.000000,0.449568
max,299.000000,20.000000,1.000000


In [4]:
wind = pd.read_csv("../data/wind.csv")
wind.describe()

,Day,Hour,Dir,Spd
count,7440.000000,7440.000000,7205.000000,7205.000000
mean,154.500000,12.500000,139.219986,0.208347
std,89.494841,6.922652,82.323134,0.126658
min,0.000000,1.000000,10.000000,0.000000
25%,77.000000,6.750000,110.000000,0.117647
50%,154.500000,12.500000,120.000000,0.205882
75%,232.000000,18.250000,160.000000,0.294118
max,309.000000,24.000000,360.000000,1.000000


In [5]:
temp = pd.read_csv("../data/temp.csv")
temp.describe()

,Day,Hour,Temp
count,7440.000000,7440.000000,7424.000000
mean,154.500000,12.500000,0.582086
std,89.494841,6.922652,0.192154
min,0.000000,1.000000,0.000000
25%,77.000000,6.750000,0.447099
50%,154.500000,12.500000,0.576792
75%,232.000000,18.250000,0.744027
max,309.000000,24.000000,1.000000


In [6]:
print(sunshine.shape, wind.shape, temp.shape)
assert wind.shape[0] == temp.shape[0]

print(sunshine.shape[0]/15, wind.shape[0]/24)
assert sunshine.shape[0]/15 == wind.shape[0]/24 - 10

(4500, 3) (7440, 4) (7440, 3)
300.0 310.0


In [7]:
import datetime

def dh2dt(_d, _h, _p=False):
    _r = datetime.datetime.strptime(f"2000-01-01 {_h-1}:00:00", "%Y-%m-%d %H:%M:%S") + datetime.timedelta(days=_d)
    if _p:
        print("dh2dt", _r)
    return _r 

def dt2dh(_dt, _p=False):
    _d = (_dt - datetime.datetime.strptime(f"2000-01-01 00:00:00", "%Y-%m-%d %H:%M:%S")).days
    _h = _dt.hour + 1
    if _p:
        print("dt2dh", _d, _h)
    return _d, _h

for _d in range(2):
    for _h in range(1, 25):
        dt2dh(dh2dt(_d, _h, _p=True), _p=True)


dh2dt 2000-01-01 00:00:00
dt2dh 0 1
dh2dt 2000-01-01 01:00:00
dt2dh 0 2
dh2dt 2000-01-01 02:00:00
dt2dh 0 3
dh2dt 2000-01-01 03:00:00
dt2dh 0 4
dh2dt 2000-01-01 04:00:00
dt2dh 0 5
dh2dt 2000-01-01 05:00:00
dt2dh 0 6
dh2dt 2000-01-01 06:00:00
dt2dh 0 7
dh2dt 2000-01-01 07:00:00
dt2dh 0 8
dh2dt 2000-01-01 08:00:00
dt2dh 0 9
dh2dt 2000-01-01 09:00:00
dt2dh 0 10
dh2dt 2000-01-01 10:00:00
dt2dh 0 11
dh2dt 2000-01-01 11:00:00
dt2dh 0 12
dh2dt 2000-01-01 12:00:00
dt2dh 0 13
dh2dt 2000-01-01 13:00:00
dt2dh 0 14
dh2dt 2000-01-01 14:00:00
dt2dh 0 15
dh2dt 2000-01-01 15:00:00
dt2dh 0 16
dh2dt 2000-01-01 16:00:00
dt2dh 0 17
dh2dt 2000-01-01 17:00:00
dt2dh 0 18
dh2dt 2000-01-01 18:00:00
dt2dh 0 19
dh2dt 2000-01-01 19:00:00
dt2dh 0 20
dh2dt 2000-01-01 20:00:00
dt2dh 0 21
dh2dt 2000-01-01 21:00:00
dt2dh 0 22
dh2dt 2000-01-01 22:00:00
dt2dh 0 23
dh2dt 2000-01-01 23:00:00
dt2dh 0 24
dh2dt 2000-01-02 00:00:00
dt2dh 1 1
dh2dt 2000-01-02 01:00:00
dt2dh 1 2
dh2dt 2000-01-02 02:00:00
dt2dh 1 3
dh2dt 2000-01

In [8]:
data = pd.merge(wind, temp, on=["Day", "Hour"], how="left")
data = pd.merge(data, sunshine, on=["Day", "Hour"], how="left")
data.fillna(-999999, inplace=True)
data["dt"] = [
    datetime.datetime.strptime(f"2000-01-01 {_h-1}:00:00", "%Y-%m-%d %H:%M:%S") + datetime.timedelta(days=_d)
    for _d, _h in zip(data["Day"], data["Hour"])
]
data["para-A"] = 2
data["para-n"] = 0.5
data

,Day,Hour,Dir,Spd,Temp,Radiation,dt,para-A,para-n
0,0,1,180.0,0.205882,0.774744,-999999.0,2000-01-01 00:00:00,2,0.5
1,0,2,170.0,0.127451,0.774744,-999999.0,2000-01-01 01:00:00,2,0.5
2,0,3,180.0,0.098039,0.764505,-999999.0,2000-01-01 02:00:00,2,0.5
3,0,4,180.0,0.058824,0.764505,-999999.0,2000-01-01 03:00:00,2,0.5
4,0,5,170.0,0.117647,0.757679,-999999.0,2000-01-01 04:00:00,2,0.5
...,...,...,...,...,...,...,...,...,...
7435,309,20,200.0,0.156863,0.757679,-999999.0,2000-11-05 19:00:00,2,0.5
7436,309,21,210.0,0.098039,0.750853,-999999.0,2000-11-05 20:00:00,2,0.5
7437,309,22,260.0,0.029412,0.744027,-999999.0,2000-11-05 21:00:00,2,0.5
7438,309,23,190.0,0.078431,0.723549,-999999.0,2000-11-05 22:00:00,2,0.5


In [9]:
from paddlets import TSDataset

data_ds = TSDataset.load_from_dataframe(
    data,
    time_col='dt',
    target_cols='Radiation',
    known_cov_cols=['Day', 'Hour', 'Dir', 'Spd', 'Temp'],
    static_cov_cols=["para-A", "para-n"],
    freq='1h',

    # max, min, avg, median, pre, back, zero
    # fill_missing_dates=True,
    # fillna_method='max' 
)
data_ds

                     Radiation  Day  Hour    Dir       Spd      Temp
dt                                                                  
2000-01-01 00:00:00  -999999.0    0     1  180.0  0.205882  0.774744
2000-01-01 01:00:00  -999999.0    0     2  170.0  0.127451  0.774744
2000-01-01 02:00:00  -999999.0    0     3  180.0  0.098039  0.764505
2000-01-01 03:00:00  -999999.0    0     4  180.0  0.058824  0.764505
2000-01-01 04:00:00  -999999.0    0     5  170.0  0.117647  0.757679
...                        ...  ...   ...    ...       ...       ...
2000-11-05 19:00:00  -999999.0  309    20  200.0  0.156863  0.757679
2000-11-05 20:00:00  -999999.0  309    21  210.0  0.098039  0.750853
2000-11-05 21:00:00  -999999.0  309    22  260.0  0.029412  0.744027
2000-11-05 22:00:00  -999999.0  309    23  190.0  0.078431  0.723549
2000-11-05 23:00:00  -999999.0  309    24  260.0  0.029412  0.720137

[7440 rows x 6 columns]

In [10]:
train_ds, testa_ds = data_ds.split(
    pd.Timestamp(dh2dt(_d=299-10, _h=24, _p=True)),
)

dh2dt 2000-10-16 23:00:00


In [11]:
train_ds, testa_ds

(                     Radiation  Day  Hour    Dir       Spd      Temp
 dt                                                                  
 2000-01-01 00:00:00  -999999.0    0     1  180.0  0.205882  0.774744
 2000-01-01 01:00:00  -999999.0    0     2  170.0  0.127451  0.774744
 2000-01-01 02:00:00  -999999.0    0     3  180.0  0.098039  0.764505
 2000-01-01 03:00:00  -999999.0    0     4  180.0  0.058824  0.764505
 2000-01-01 04:00:00  -999999.0    0     5  170.0  0.117647  0.757679
 ...                        ...  ...   ...    ...       ...       ...
 2000-11-05 19:00:00        NaN  309    20  200.0  0.156863  0.757679
 2000-11-05 20:00:00        NaN  309    21  210.0  0.098039  0.750853
 2000-11-05 21:00:00        NaN  309    22  260.0  0.029412  0.744027
 2000-11-05 22:00:00        NaN  309    23  190.0  0.078431  0.723549
 2000-11-05 23:00:00        NaN  309    24  260.0  0.029412  0.720137
 
 [7440 rows x 6 columns],
                      Radiation  Day  Hour    Dir       Spd   

In [12]:
model = make_ml_model(
    in_chunk_len=3,
    out_chunk_len=1,
    model_class=RandomForestRegressor
)

In [13]:
model.fit(train_data=train_ds)

In [14]:
train_pr = model.recursive_predict(
    tsdataset=train_ds, 
    predict_length=20 * 24
)

from sklearn.metrics import mean_squared_error
train_mse = mean_squared_error(
    data_ds.to_dataframe().tail(20*24).head(10*24)["Radiation"].to_numpy(),
    train_pr.to_numpy()[:,0][:10*24]
)
print(f"{train_mse:.4f}")


0.0216


In [15]:
_result = train_pr.to_dataframe()
_result["_d"] = [dt2dh(i)[0] for i in _result.index]
_result["_h"] = [dt2dh(i)[1] for i in _result.index]
_result = _result[(_result["_d"] >= 300) & (_result["_h"] >= 6) & (_result["_h"] <= 20)]
_result.to_csv("result.csv", index=False)
_result

,Radiation,_d,_h
2000-10-27 05:00:00,0.000007,300,6
2000-10-27 06:00:00,0.011873,300,7
2000-10-27 07:00:00,0.072190,300,8
2000-10-27 08:00:00,0.166138,300,9
2000-10-27 09:00:00,0.328588,300,10
...,...,...,...
2000-11-05 15:00:00,0.359424,309,16
2000-11-05 16:00:00,0.170980,309,17
2000-11-05 17:00:00,0.034640,309,18
2000-11-05 18:00:00,0.005245,309,19
